# Generative AI Project

In [ ]:
!pip install transformers torch

In [ ]:
import torch
import transformers

In [ ]:
model = transformers.AutoModelForCausalLM.from_pretrained('gpt2')

In [ ]:
def generate_text(prompt):
    inputs = tokenizer(prompt, return_tensors='pt')
    outputs = model.generate(inputs['input_ids'], max_length=100)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)